In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from tqdm.notebook import trange, tqdm

# Загрузка данных

In [3]:
rep_df = pd.read_csv('hg38_2020_rmsk.bed', sep='\t', header=None, names=['chrom', 'start', 'end', 'name', '-', 'strand'])
rep_df['point'] = round(rep_df.start + (rep_df.end - rep_df.start) / 2, 0)
rep_df.head(2)

,chrom,start,end,name,-,strand,point
0,chr1,67108750,67109048,L1P5,262,+,67108899.0
1,chr1,8388317,8388618,AluYb8,373,-,8388468.0


In [4]:
superfam_list = []
for family in rep_df.name.tolist():
    if 'Alu' in family:
        superfamily = 'Alu'
    elif 'L1' in family:
        superfamily = 'L1'
    elif 'L2' in family:
        superfamily = 'L2'
    elif 'MIR' in family:
        superfamily = 'MIR'
    elif 'LTR' in family:
        superfamily = 'LTR'
    elif 'L3' in family:
        superfamily = 'L3'
    elif 'MLT' in family:
        superfamily = 'MLT'
    elif 'MER' in family:
        superfamily = 'MER'
    elif 'THE' in family:
        superfamily = 'THE'
    else:
        superfamiy = family
    superfam_list.append(superfamily)
    
rep_df['superfamily'] = superfam_list

In [5]:
rep_df.name.value_counts().iloc[:10]#.index[0]

name
MIRb     231470
MIR      191908
AluSx    168481
L2c      161344
L2b      143963
L2a      141633
AluJb    139520
AluJr    108744
MIRc     102225
AluSg    100494
Name: count, dtype: int64

In [6]:
rep_df.superfamily.value_counts().iloc[:10]#.index[0]

superfamily
Alu    1593422
L1     1379417
MIR     862336
L2      774530
MER     567382
MLT     396428
LTR     299925
THE      74205
L3       62387
Name: count, dtype: int64

In [7]:
lp_df_1 = pd.read_csv('dataset1_simplified_LPs.tsv', sep='\t')
lp_df_2 = pd.read_csv('dataset2_simplified_LPs.tsv', sep='\t')
lp_df_2.head(2)

,chr,LP1,LP2
0,chrY,8783767,8783820
1,chrY,386892,386872


# Анализ

In [8]:
flang_length = 50000
bin_size = 10000
bin_num = int(flang_length * 2 / bin_size)

In [9]:
def get_bin_list(lp_df, el_df):
    
    global_bin_list_lp1 = []
    for lp in tqdm(lp_df.LP1.tolist()):

        start = lp - flang_length
        end = lp + flang_length

        bin_list = []
        for i in range(start, end, bin_size):
            c = el_df[(el_df.start >= i) & (el_df.end <= i + bin_size)].shape[0]
            bin_list.append(c)
        global_bin_list_lp1.append(bin_list)
        
    global_bin_list_lp2 = []
    for lp in tqdm(lp_df.LP2.tolist()):

        start = lp - flang_length
        end = lp + flang_length

        bin_list = []
        for i in range(start, end, bin_size):
            c = el_df[(el_df.start >= i) & (el_df.end <= i + bin_size)].shape[0]
            bin_list.append(c)
        global_bin_list_lp2.append(bin_list)
        
    return global_bin_list_lp1, global_bin_list_lp2

In [10]:
for el in rep_df.superfamily.value_counts().iloc[:9].index:
    el_df = rep_df[rep_df['superfamily'] == el]
    el_df_plus = rep_df[(rep_df['superfamily'] == el) & (rep_df['strand'] == '+')]
    el_df_minus = rep_df[(rep_df['superfamily'] == el) & (rep_df['strand'] == '-')]
    
    # Get bin lists
    global_bin_list_lp1_ds1_plus, global_bin_list_lp2_ds1_plus = get_bin_list(lp_df_1, el_df_plus)
    global_bin_list_lp1_ds2_plus, global_bin_list_lp2_ds2_plus = get_bin_list(lp_df_2, el_df_plus)
    global_bin_list_lp1_ds1_minus, global_bin_list_lp2_ds1_minus = get_bin_list(lp_df_1, el_df_minus)
    global_bin_list_lp1_ds2_minus, global_bin_list_lp2_ds2_minus = get_bin_list(lp_df_2, el_df_minus)
    
    
    normalization_score = round(pd.DataFrame(global_bin_list_lp1_ds1_plus).mean().mean() / 2, 1)
    mean_list_lp1_ds1_plus = (pd.DataFrame(global_bin_list_lp1_ds1_plus).mean() + normalization_score).tolist()
    mean_list_lp2_ds1_plus = (pd.DataFrame(global_bin_list_lp2_ds1_plus).mean() + normalization_score).tolist()
    mean_list_lp1_ds2_plus = pd.DataFrame(global_bin_list_lp1_ds2_plus).mean().tolist()
    mean_list_lp2_ds2_plus = pd.DataFrame(global_bin_list_lp2_ds2_plus).mean().tolist()
    
    mean_list_lp1_ds1_minus = (pd.DataFrame(global_bin_list_lp1_ds1_minus).mean() + normalization_score).tolist()
    mean_list_lp2_ds1_minus = (pd.DataFrame(global_bin_list_lp2_ds1_minus).mean() + normalization_score).tolist()
    mean_list_lp1_ds2_minus = pd.DataFrame(global_bin_list_lp1_ds2_minus).mean().tolist()
    mean_list_lp2_ds2_minus = pd.DataFrame(global_bin_list_lp2_ds2_minus).mean().tolist()
    
    # Create density dataframe
    values = mean_list_lp1_ds1_plus + mean_list_lp2_ds1_plus + mean_list_lp1_ds2_plus + mean_list_lp2_ds2_plus + mean_list_lp1_ds1_minus + mean_list_lp2_ds1_minus + mean_list_lp1_ds2_minus + mean_list_lp2_ds2_minus
    names = [f'LP1_Dataset1_YS+{normalization_score}'] * bin_num + [f'LP2_Dataset1_YS+{normalization_score}'] * bin_num + ['LP1_Dataset2'] * bin_num + ['LP2_Dataset2'] * bin_num + [f'LP1_Dataset1_YS+{normalization_score}'] * bin_num + [f'LP2_Dataset1_YS+{normalization_score}'] * bin_num + ['LP1_Dataset2'] * bin_num + ['LP2_Dataset2'] * bin_num
    strand = ['+'] * bin_num * 4 + ['-'] * bin_num * 4
    coord = list(range(-50000, 50000, 1000)) * 8
    df = pd.DataFrame({'coord':coord, 'values':values, 'names':names, 'strand':strand})
    
    # Create plot
    sns.set_theme(rc={'figure.figsize':(15,8)}, style="whitegrid", font_scale=1.2)
    ax = None
    fig, ax = plt.subplots(1, 1)
    sns.lineplot(data=df, x=coord, y='values', hue='names', style="strand", ax=ax)
    ax.set(xlabel='Distance from ligation point', ylabel='Density', title=f'{el} mobile element density around LP')
    ax.set_xticks(range(-50000, 50000, 10000));
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

    # Save plot
    fig = ax.get_figure()
    fig.savefig(f'{el}.png',pad_inches=0.5, bbox_inches='tight')
    plt.close()
    
    df.to_csv(f'{el}_plot_data.csv', index=False)

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3383 [00:00<?, ?it/s]

  0%|          | 0/3383 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3383 [00:00<?, ?it/s]

  0%|          | 0/3383 [00:00<?, ?it/s]

ValueError: All arrays must be of the same length

In [14]:
plt.close()

normalization_score = pd.DataFrame(global_bin_list_lp1_ds1_plus).mean().mean() / 2
mean_list_lp1_ds1_plus = (pd.DataFrame(global_bin_list_lp1_ds1_plus).mean() + normalization_score).tolist()
mean_list_lp2_ds1_plus = (pd.DataFrame(global_bin_list_lp2_ds1_plus).mean() + normalization_score).tolist()
mean_list_lp1_ds2_plus = pd.DataFrame(global_bin_list_lp1_ds2_plus).mean().tolist()
mean_list_lp2_ds2_plus = pd.DataFrame(global_bin_list_lp2_ds2_plus).mean().tolist()

mean_list_lp1_ds1_minus = (pd.DataFrame(global_bin_list_lp1_ds1_minus).mean() + normalization_score).tolist()
mean_list_lp2_ds1_minus = (pd.DataFrame(global_bin_list_lp2_ds1_minus).mean() + normalization_score).tolist()
mean_list_lp1_ds2_minus = pd.DataFrame(global_bin_list_lp1_ds2_minus).mean().tolist()
mean_list_lp2_ds2_minus = pd.DataFrame(global_bin_list_lp2_ds2_minus).mean().tolist()

values = mean_list_lp1_ds1_plus + mean_list_lp2_ds1_plus + mean_list_lp1_ds2_plus + mean_list_lp2_ds2_plus + mean_list_lp1_ds1_minus + mean_list_lp2_ds1_minus + mean_list_lp1_ds2_minus + mean_list_lp2_ds2_minus
names = ['Exbor1_Dataset1'] * bin_num + ['Exbor2_Dataset1'] * bin_num + ['Exbor1_Dataset2'] * bin_num + ['Exbor2_Dataset2'] * bin_num + ['Exbor1_Dataset1'] * bin_num + ['Exbor2_Dataset1'] * bin_num + ['Exbor1_Dataset2'] * bin_num + ['Exbor2_Dataset2'] * bin_num
strand = ['+'] * bin_num * 4 + ['-'] * bin_num * 4
coord = list(range(-50000, 50000, 1000)) * 8
df = pd.DataFrame({'coord':coord, 'values':values, 'names':names, 'strand':strand})

# Create plot
sns.set_theme(rc={'figure.figsize':(15,8)}, style="whitegrid", font_scale=1.2)
ax = None
fig, ax = plt.subplots(1, 1)
sns.lineplot(data=df, x=coord, y='values', hue='names', style="strand", ax=ax)
ax.set(xlabel='Distance from ligation point', ylabel='Density', title=f'{el} mobile element density around LP')
ax.set_xticks(range(-50000, 50000, 10000));
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5));

# Save plot
fig = ax.get_figure()
fig.savefig(f'{el}.png',pad_inches=0.5, bbox_inches='tight')
plt.close()

In [36]:
pd.DataFrame(global_bin_list_lp1_ds2_minus).mean().mean() * 5

0.6630653266331659

In [14]:
rep_df.name.value_counts().iloc[19:20]

name
A-rich    52142
Name: count, dtype: int64

In [18]:
rep_df[rep_df.name == 'A-rich'].strand.value_counts()

strand
+    52142
Name: count, dtype: int64